[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ZachDorm/basal_and_bark/blob/main/docs/examples/fia_db.ipynb)

In [ ]:
# import requests
# import pandas as pd
# import geopandas as gp
import basal_and_bark
from basal_and_bark import basal_and_bark as basal
import ipyleaflet
# from ipyleaflet import Marker, Popup, WidgetControl, GeoData
from ipyleaflet import WidgetControl
import ipywidgets
from ipywidgets import HTML, widgets

In [ ]:
leaflet_map = basal.Map(center = [40,-100], zoom = 4, test = "test", scroll_wheel_zoom = True)

In [ ]:
leaflet_map.add_layer_control()

In [ ]:
leaflet_map

In [ ]:
# leaflet_map.add_shp("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2022_us_state.zip")

In [ ]:
tn_counties_gd = leaflet_map.addRefData()


In [ ]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_widget.clear_output()
basemap_ctrl = WidgetControl(widget=output_widget, position='bottomright')
leaflet_map.add_control(basemap_ctrl)

dropdown = widgets.Dropdown(
            options = ["2018", "2019", "2020", "2021"], 
            value="2020",
            description='Year',
            )

with output_widget:
            display(dropdown)

In [ ]:
def handle_click(**kwargs):

    if kwargs.get('type') == 'click':
        output_widget.clear_output()
        #leaflet_map.add_layer(Marker(location=kwargs.get('coordinates')))
        # latlon = kwargs.get('coordinates')

        gdf = leaflet_map.makePointsFromClick(kwargs.get('coordinates'))

        county = leaflet_map.findInt(tn_counties_gd, gdf)
        # for i in range(1,len(tn_counties_gd['geometry']),1):
        #     test = tn_counties_gd['geometry'][i].contains(gdf['geometry'])

        #     county = 'name'

        #     if test[0]:
        #         county = tn_counties_gd['STUSPS'][i]
        with output_widget:
            display(dropdown)
            print(county)
        print(leaflet_map.getAPIdata(county, dropdown.value))#['ESTIMATE'])
        print("The above table are estimates for 6 different categories of land, as specified by USFS: "+ 
              "'Timberland', 'Reserved', 'Other Forestland', 'Nonforest', 'Non-Census Water', 'Census Water', 'Other'")
        print("The table below contains summary statistics of the canopy cover in the Hansen et al Forest Change dataset.")
        print(leaflet_map.addGEEData(county))

leaflet_map.on_interaction(handle_click)

In [ ]:
output_widget.widgets[len(output_widget.widgets)-1]

In [ ]:
output_widget.widgets.values()[len(output_widget.widgets.values())-1]

In [ ]:
keys = output_widget.widgets.keys()
keys[len(keys)-1]

In [ ]:
output_widget.widgets["8561c5197658434691ce5b727c699ad0"].value